In [1]:
import os, sys, warnings, operator, itertools
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error as mse
def rmse(pred, true): return np.sqrt(mse(true, pred))

In [2]:
read_path = './dataset/origin/'
write_path = './dataset/submission/'
sample = pd.read_csv(read_path+'sample.csv')
train = pd.read_csv(read_path+'train.csv')
test = pd.read_csv(read_path+'test_mini.csv')

## 타겟과 로스

1. 진짜 타겟(12개의 합)을 타겟으로 삼는다 -> lgb로 망함 ㅋ
2. 12개를 각각 타겟으로 삼는다 -> lgb로 망함 ㅋ
1. 그냥 2014년도 자료를 제출한다 -> 316.439768872516
3. 자살자 수 변화율을 타겟으로 삼는다 -> 200.62501063108937
4. 자살자 수 변화율12 -> 196.00703758110848

---
5. 자살율 변화율을 타겟으로 삼는다 -> 163.0990776087103
6. 자살율 변화율12 -> 158.7807797764072
1. 자살자 수 변화량 (처음에 했던 거에서 조금 진보) -> 219.307695339681
1. 자살자 수 변화량 12 -> 219.307695339681
1. 자살율 변화량 -> 자살율이 음수 나와서 fail
1. 자살율 변화량 12 -> 자살율이 음수 나와서 fail

## 자살율 변화율

In [31]:
train['suicides_rate'] = train['suicides_no']/train['population']*1e+10
testCountries = test['country'].unique().tolist()
train_pure = train[train['country'].apply(lambda x: x in testCountries)]
train_2014 = train_pure[train_pure['year']==2014]
train_2013 = train_pure[train_pure['year']==2013]
ukraine = train_pure[(train_pure['country']=='Ukraine') & (train_pure['year']==2012)]
train_2013 = train_2013.append(ukraine)
cols = ['country', 'sex', 'age']
train_2014 = train_2014.sort_values(by=cols)
train_2013 = train_2013.sort_values(by=cols)
test_2015 = test.sort_values(by=cols)
train_2013 = train_2013.reset_index(drop=True)
train_2014 = train_2014.reset_index(drop=True)
test_2015 = test_2015.reset_index(drop=True)
print(sum(train_2014[cols].values != train_2013[cols].values))
print(sum(train_2014[cols].values != test_2015[cols].values))

[0 0 0]
[0 0 0]


### 12개 각각 구하기

In [23]:
cols = ['country', 'suicides_no', 'population', 'suicides_rate', 'year']
train_2014 = train_2014[cols]
train_2013 = train_2013[cols]
train_2014.columns = cols[:-4]+[col+'_2014' for col in cols[1:]]
train_2013.columns = cols[:-4]+[col+'_2013' for col in cols[1:]]
train_20134 = pd.concat([train_2014, train_2013[[col+'_2013' for col in cols[1:]]]],axis=1)

In [24]:
def diff_btw_2013_2014(x):
    _2014 = x['suicides_rate_2014']
    _2013 = x['suicides_rate_2013']
    try : return (_2014-_2013)/_2013+1
    except ZeroDivisionError : return np.nan
train_20134['diff_btw_2013_2014'] = train_20134.apply(diff_btw_2013_2014, axis=1)

In [25]:
def suicides_rate_2015(x):
    _2014 = x['suicides_rate_2014']
    rate = x['diff_btw_2013_2014']
    if pd.isna(rate): return _2014
    else : return rate*_2014
train_20134['suicides_rate_2015'] = train_20134.apply(suicides_rate_2015, axis=1)

In [26]:
test_2015 = pd.concat([test_2015, train_20134[['suicides_rate_2015']]],axis=1)

In [27]:
test_2015['y_pred'] = test_2015['suicides_rate_2015']*test_2015['population']/1e+10

In [28]:
test_2015_agg = test_2015.groupby('country')['y_pred'].agg('sum').reset_index()

In [29]:
test_2015_agg.to_csv(write_path+'03_suicides_rate_diff_btw_2013_2014_cbc.csv', index=False)

### 합쳐서 구하기

In [32]:
cols = ['country', 'suicides_no', 'population', 'suicides_rate', 'year']
train_2014 = train_2014[cols]
train_2013 = train_2013[cols]

aggs = {
    'suicides_no': 'sum',
    'population': 'sum',
}
train_2014_agg = train_2014.groupby('country').agg(aggs).reset_index()
train_2013_agg = train_2013.groupby('country').agg(aggs).reset_index()
test_2015_agg = test_2015.groupby('country')['population'].agg('sum').reset_index()

In [33]:
train_2014_agg['suicides_rate_2014'] = train_2014_agg['suicides_no']*1e+10/train_2014_agg['population']
train_2013_agg['suicides_rate_2013'] = train_2013_agg['suicides_no']*1e+10/train_2013_agg['population']

In [35]:
train_2015 = pd.concat([test_2015_agg, train_2014_agg[['suicides_rate_2014']], train_2013_agg[['suicides_rate_2013']]],axis=1)
train_2015['diff_btw_2013_2014'] = train_2015.apply(diff_btw_2013_2014, axis=1)
train_2015['suicides_rate_2015'] = train_2015.apply(suicides_rate_2015, axis=1)
train_2015['y_pred'] = train_2015['suicides_rate_2015']*train_2015['population']/1e+10

In [36]:
train_2015.to_csv(write_path+'03_suicides_rate_diff_btw_2013_2014_total.csv', index=False)

## 자살자 수 변화량

In [42]:
testCountries = test['country'].unique().tolist()
train_pure = train[train['country'].apply(lambda x: x in testCountries)]
train_2014 = train_pure[train_pure['year']==2014]
train_2013 = train_pure[train_pure['year']==2013]
ukraine = train_pure[(train_pure['country']=='Ukraine') & (train_pure['year']==2012)]
train_2013 = train_2013.append(ukraine)
cols = ['country', 'sex', 'age']
train_2014 = train_2014.sort_values(by=cols)
train_2013 = train_2013.sort_values(by=cols)
test_2015 = test.sort_values(by=cols)
train_2013 = train_2013.reset_index(drop=True)
train_2014 = train_2014.reset_index(drop=True)
test_2015 = test_2015.reset_index(drop=True)
print(sum(train_2014[cols].values != train_2013[cols].values))
print(sum(train_2014[cols].values != test_2015[cols].values))

[0 0 0]
[0 0 0]


### 12개 각각 구하기

In [11]:
cols = ['country', 'suicides_no',]
train_2014 = train_2014[cols]
train_2013 = train_2013[cols]
train_2014.columns = cols[:-1]+[col+'_2014' for col in cols[1:]]
train_2013.columns = cols[:-1]+[col+'_2013' for col in cols[1:]]
train_2015 = pd.concat([test_2015, train_2014[[col+'_2014' for col in cols[1:]]], train_2013[[col+'_2013' for col in cols[1:]]]],axis=1)
print(train_2015.shape)

(744, 13)


In [13]:
train_2015['suicides_no_diff_btw_2013_2014'] = train_2015['suicides_no_2014']-train_2015['suicides_no_2013']
train_2015['y_pred'] = train_2015['suicides_no_2014']+train_2015['suicides_no_diff_btw_2013_2014']
train_2015_agg = train_2015.groupby('country')['y_pred'].agg('sum').reset_index()

In [15]:
train_2015_agg.to_csv(write_path+'03_suicides_no_amount_diff_cbc.csv', index=False)

### 12개 합쳐서 구하기

In [43]:
train_2014_agg = train_2014.groupby('country')['suicides_no'].agg('sum').reset_index()
train_2013_agg = train_2013.groupby('country')['suicides_no'].agg('sum').reset_index()
test_2015_agg = test_2015.groupby('country')['sex'].agg('count').reset_index()
cols = ['country', 'suicides_no',]
train_2014_agg.columns = cols[:-1]+[col+'_2014' for col in cols[1:]]
train_2013_agg.columns = cols[:-1]+[col+'_2013' for col in cols[1:]]
train_2015 = pd.concat([test_2015_agg, train_2014_agg[[col+'_2014' for col in cols[1:]]], train_2013_agg[[col+'_2013' for col in cols[1:]]]],axis=1)

In [44]:
train_2015['diff_btw_2013_2014'] = train_2015['suicides_no_2014'] - train_2015['suicides_no_2013']
train_2015['y_pred'] = train_2015['suicides_no_2014']+train_2015['diff_btw_2013_2014']
train_2015.to_csv(write_path+'03_suicides_no_amount_diff_total.csv', index=False)

In [47]:
sum(train_2015['y_pred']==train_2015_agg['y_pred'])

62

## 결론 : cbc>total, 자살율 변화율 >> 자살자 수 변화율 > 자살자 수 변화량 >>> base 316.44